## Spotify API Class

In [ ]:
import base64
import requests
import datetime
from urllib.parse import urlencode

class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_id = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self,client_id,client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None :
            raise Exception('Please specify your Client id and secret first')
        client_creds = base64.urlsafe_b64encode(f'{client_id}:{client_secret}'.encode())
        return client_creds.decode()
        
    def get_token_headers(self):
        client_creds = self.get_client_credentials()
        return {
            'Authorization': f'Basic {client_creds}'
            }
    
    def get_token_data(self):
        return {
            'grant_type': 'client_credentials'
            }
    
    def perform_authorization(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data = token_data, headers = token_headers)
        if r.status_code not in range(200,299):
            raise Exception('Could not authen client')
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in']
        expires = now + datetime.timedelta(seconds = expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

    def get_access_token(self): 
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_authorization()
            return self.get_access_token()
        elif token == None:
            self.perform_authorization()
            return self.get_access_token()
        return token 
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            'Authorization': f'Bearer {access_token}'
            }
        return headers
                    
    def get_album(self,_id):
        base_url = 'https://api.spotify.com'
        endpoint = f'{base_url}/v1/albums/{_id}'
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers = headers)
        if r.status_code not in range (200,299):
            return {}
        return r.json()
    
    def get_artist(self,_id):
        base_url = 'https://api.spotify.com'
        endpoint = f'{base_url}/v1/artists/{_id}'
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers = headers)
        if r.status_code not in range (200,299):
            return {}
        return r.json()
    
    def get_tracks_audiofeatures(self,_ids):
        #comma seperated list of ids
        base_url = 'https://api.spotify.com/v1/audio-features'
        endpoint = f'{base_url}?{_ids}'
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers = headers)
        if r.status_code not in range (200,299):
            return {}
        return r.json()
        
    def base_search(self, query_paramas):
        headers = self.get_resource_header()
        endpoint = 'https://api.spotify.com/v1/search'
        lookupurl = f'{endpoint}?{query_paramas}'
        r = requests.get(lookupurl, headers = headers)
        if r.status_code not in range (200,299):
            return {}
        return r.json()

    def search(self,query = None, operator = None, operator_query = None, search_type = 'artist', limit = 1):
        #query is a dictionary
        if query == None:
            raise Exception ('A Query is required')
        if isinstance(query, dict):
            query = ' '.join([f'{k}:{v}' for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == 'or' or operator.lower() == 'not':
                operator = operator.upper()
                if isinstance(operator_query,str):
                    query = f'{query} {operator} {operator_query}'
        query_params = urlencode({'q': query ,'type': search_type.lower()})
        return self.base_search(query_params)

In [ ]:
app = SpotifyAPI('a9e59a497b4248aab60e35784bfe45ca','7fdec693cd9943188bd5a63e2d87261b')

# Getting the IDs of the Songs




In [ ]:
  app.search({'track':'','artist':''}, search_type = 'track')

# Song Features 






In [ ]:
import numpy as np
import pandas as pd

Grammy2020dict = app.get_tracks_audiofeatures('ids=2qzUpSVI4NnPyWxbXwumTj,6vaSStNN5NX4nJ4QbRY3S0,7ytR5pFWmSjzHJIeQkgog4,3Dv1eDb0MEgF93GpLXlucZ,3ZCTVFBt2Brf31RLEnCkWJ,3PfIrDoz19wz7qK7tYeu62,21jGcNKet2qwijlDFuPiPb,5v4GgrXPMghOnBBLmveLac')
Grammy2020list = np.array([(v) for k,v in Grammy2020dict.items()])
Grammy2020 = []
for i in range(0,8):
   Grammy2020 = np.append(Grammy2020, np.array([(v) for k,v in Grammy2020list[0][i].items()]))
Grammy2020 = Grammy2020.reshape((8,18))
Grammy2020 = Grammy2020[:,0:11]

Grammy2019dict = app.get_tracks_audiofeatures('ids=2Fxmhks0bxGSBdJ92vM42m,0d6bZwEAi1XblGRhnzPfgC,6ocbgoVGwYJhOv1GgI9NsF,4IablJ6SqVNGY4vrseyKxu,6TqXcAFInzjp0bODyvrWEq,0F7FA14euOIX8KcbEturGH,4JazDcr295JjClPNnks5WL,3KkXRkHbMCARz0aVfEt68P')
Grammy2019list = np.array([(v) for k,v in Grammy2019dict.items()])
Grammy2019 = []
for i in range(0,8):
   Grammy2019 = np.append(Grammy2019, np.array([(v) for k,v in Grammy2019list[0][i].items()]))
Grammy2019 = Grammy2019.reshape((8,18))
Grammy2019 = Grammy2019[:,0:11]

Grammy2018dict = app.get_tracks_audiofeatures('ids=0b9oOr2ZgvyQu88wzixux9,58q2HKrzhC3ozto2nDdN4z,18k8IlS52uxRWObykDUZFG,6DCZcSspjsKoFjzjrWoCdn,2VxeLyX666F8uXCJ0dZF8B,3GCdLUSnKSMJhs4Tj6CV3s,0e7ipj03S05BNilyu5bRzt,09IStsImFySgyp0pIQdqAc')
Grammy2018list = np.array([(v) for k,v in Grammy2018dict.items()])
Grammy2018 = []
for i in range(0,8):
   Grammy2018 = np.append(Grammy2018, np.array([(v) for k,v in Grammy2018list[0][i].items()]))
Grammy2018 = Grammy2018.reshape((8,18))
Grammy2018 = Grammy2018[:,0:11]

Grammy2017dict = app.get_tracks_audiofeatures('ids=0KKkJNfGyhkQ5aFogxQAPU,0wXuerDYiBnERgIpbb3JBR,6rPO02ozF3bM7NnOV4h6s2,6JpN5w95em8SODPiM7W2PH,7KXjTSCq5nL1LoYtL7XAwS')
Grammy2017list = np.array([(v) for k,v in Grammy2017dict.items()])
Grammy2017 = []
for i in range(0,5):
   Grammy2017 = np.append(Grammy2017, np.array([(v) for k,v in Grammy2017list[0][i].items()]))
Grammy2017 = Grammy2017.reshape((5,18))
Grammy2017 = Grammy2017[:,0:11]

Grammy2016dict = app.get_tracks_audiofeatures('ids=4sPmO7WMQUAf45kwMOtONw,6g0Orsxv6glTJCt4cHsRsQ,5kqIPrATaCc2LqxVWzQGbk,72TFWvU3wUYdUuxejTTIzt,3CRDbSIZ4r5MsZ0YwxuEkn')
Grammy2016list = np.array([(v) for k,v in Grammy2016dict.items()])
Grammy2016 = []
for i in range(0,5):
   Grammy2016 = np.append(Grammy2016, np.array([(v) for k,v in Grammy2016list[0][i].items()]))
Grammy2016 = Grammy2016.reshape((5,18))
Grammy2016 = Grammy2016[:,0:11]

Spotify2020dict = app.get_tracks_audiofeatures('ids=0VjIjW4GlUZAMYd2vXMi3b,2XU0oxnq2qxCpomAAuJY8K,0nbXyq5TXYPCO7pr3N8S4I,2Wo6QQD1KMDWeFkkjLqwx5,3PfIrDoz19wz7qK7tYeu62')
Spotify2020list = np.array([(v) for k,v in Spotify2020dict.items()])
Spotify2020 = []
for i in range(0,5):
   Spotify2020 = np.append(Spotify2020, np.array([(v) for k,v in Spotify2020list[0][i].items()]))
Spotify2020 = Spotify2020.reshape((5,18))
Spotify2020 = Spotify2020[:,0:11]

Spotify2019dict = app.get_tracks_audiofeatures('ids=0TK2YIli7K1leLovkQiNik,2Fxmhks0bxGSBdJ92vM42m,3KkXRkHbMCARz0aVfEt68P,6ocbgoVGwYJhOv1GgI9NsF,0F7FA14euOIX8KcbEturGH')
Spotify2019list = np.array([(v) for k,v in Spotify2019dict.items()])
Spotify2019 = []
for i in range(0,5):
   Spotify2019 = np.append(Spotify2019, np.array([(v) for k,v in Spotify2019list[0][i].items()]))
Spotify2019 = Spotify2019.reshape((5,18))
Spotify2019 = Spotify2019[:,0:11]

Spotify2018dict = app.get_tracks_audiofeatures('ids=6DCZcSspjsKoFjzjrWoCdn,3ee8Jmje8o58CHK66QrVC2,0e7ipj03S05BNilyu5bRzt,3swc6WTsr7rl9DqQKQA55C,2G7V7zsVDxg1yRsu7Ew9RJ')
Spotify2018list = np.array([(v) for k,v in Spotify2018dict.items()])
Spotify2018 = []
for i in range(0,5):
   Spotify2018 = np.append(Spotify2018, np.array([(v) for k,v in Spotify2018list[0][i].items()]))
Spotify2018 = Spotify2018.reshape((5,18))
Spotify2018 = Spotify2018[:,0:11]

Spotify2017dict = app.get_tracks_audiofeatures('ids=7qiZfU4dY1lWllzX7mPBI3,6rPO02ozF3bM7NnOV4h6s2,6habFhsOp2NvshLv26DqMb,6RUKPb4LETWmmr3iAEQktW,608a1wIsSd5KzMEqm1O7w3')
Spotify2017list = np.array([(v) for k,v in Spotify2017dict.items()])
Spotify2017 = []
for i in range(0,5):
   Spotify2017 = np.append(Spotify2017, np.array([(v) for k,v in Spotify2017list[0][i].items()]))
Spotify2017 = Spotify2017.reshape((5,18))
Spotify2017 = Spotify2017[:,0:11]

Spotify2016dict = app.get_tracks_audiofeatures('ids=1zi7xx7UVEFkmKfv06H8x0,0vbtURX4qv1l7besfwmnD8,1i1fxkWeaMmKEB4T7zqbzK,72TFWvU3wUYdUuxejTTIzt,3S4px9f4lceWdKf0gWciFu')
Spotify2016list = np.array([(v) for k,v in Spotify2016dict.items()])
Spotify2016 = []
for i in range(0,5):
   Spotify2016 = np.append(Spotify2016, np.array([(v) for k,v in Spotify2016list[0][i].items()]))
Spotify2016 = Spotify2016.reshape((5,18))
Spotify2016 = Spotify2016[:,0:11]

Headings = []
for i in range(0,1):
    Headings = np.append(Headings, np.array([(k) for k,v in Grammy2020list[0][i].items()]))
Headings = Headings.reshape((1,18))
Headings = Headings[:1,0:11]

columns = [f'{i}' for i in Headings[0]]
index1 = ['Black Parade - Beyonce','Colors - Black Pumas','Rockstar - DaBaby Featuring Roddy Ricch','Say So - Doja Cat','Everything I Wanted - Billie Eilish','Don’t Start Now - Dua Lipa','Circles - Post Malone','Savage - Megan Thee Stallion Featuring Beyoncé']
index2 = ['Bad Guy – Billie Eilish','Hey, Ma - Bon Iver','7 Rings – Ariana Grande','Hard Place – H.E.R.','Talk – Khalid ft Disclosure','Old Town Road – Lil Nas X Featuring Billy Ray Cyrus','Truth Hurts – Lizzo','Sunflower – Post Malone & Swae Lee']
index3 = ['This Is America – Childish Gambino','I Like It – Cardi B, Bad Bunny & J Balvin','The Joke – Brandi Carlile','God\'s Plan – Drake','Shallow – Lady Gaga & Bradley Cooper','All the Stars – Kendrick Lamar & SZA','Rockstar – Post Malone Featuring 21 Savage','The Middle – Zedd, Maren Morris and Grey']
index4 = ['That\'s What I Likea – Bruno Mars','Redbone – Childish Gambino','Despacito – Luis Fonsi and Daddy Yankee Featuring Justin Bieber','The Story of O.J. – Jay-Z','Humble – Kendrick Lamar']
index5 = ['Hello – Adele','Formation – Beyoncé','7 Years – Lukas Graham','Work – Rihanna featuring Drake','Stressed Out – Twenty One Pilots']
index6 = ['Blinding Light - The Weekend','Dance Monkey - Tones And I','The Box - Roddy Ricch','Roses (Imanbek Remix) - Imanbek and SAINt JHN','Don’t Start Now - Dua Lipa']
index7 = ['Señorita – Camila Cabello, Shawn Mendes','Bad Guy – Billie Eilish','Sunflower – Post Malone, Swae Lee','7 Rings – Ariana Grande','Old Town Road – Remix – Lil Nas X, Billy Ray Cyrus']
index8 = ['God’s Plan – Drake','SAD! – XXXTENTACION','Rockstar  – Post Malone Featuring 21 Savage','Psycho  – Post Malone Featuring Ty Dolla $ign','In My Feelings – Drake']
index9 = ['Shape of You - Ed Sheeran','Despacito (Remix) - Luis Fonsi, Daddy Yankee and Justin Bieber','Despacito - Luis Fonsi Featuring Daddy Yankee','Something Just Like This - The Chainsmokers Featuring Coldplay','I’m the One - DJ Khaled, Justin Bieber, Chance the Rapper, Quavo, Lil Wayne']
index10 = ['One Dance – Drake Featuring WizKid and Kyla','I Took A Pill in Ibiza – Seeb Remix – Mike Posner','Don’t Let Me Down – The Chainsmokers Featuring Daya','Work – Rihanna Featuring Drake','Cheap Thrills – Sia']

df1 = pd.DataFrame(Grammy2020, columns=columns, index=index1)
df2 = pd.DataFrame(Grammy2019, columns=columns, index=index2)
df3 = pd.DataFrame(Grammy2018, columns=columns, index=index3)
df4 = pd.DataFrame(Grammy2017, columns=columns, index=index4)
df5 = pd.DataFrame(Grammy2016, columns=columns, index=index5)
df6 = pd.DataFrame(Spotify2020, columns=columns, index=index6)
df7 = pd.DataFrame(Spotify2019, columns=columns, index=index7)
df8 = pd.DataFrame(Spotify2018, columns=columns, index=index8)
df9 = pd.DataFrame(Spotify2017, columns=columns, index=index9)
df10 = pd.DataFrame(Spotify2016, columns=columns, index=index10)

#Table View

In [ ]:
df1

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
Black Parade - Beyonce,0.824,0.555,1,-5.817,1,0.05,0.00337,6.42e-06,0.217,0.0611,120.002
Colors - Black Pumas,0.391,0.692,1,-6.543,0,0.0462,0.0946,1.58e-05,0.191,0.56,180.008
Rockstar - DaBaby Featuring Roddy Ricch,0.746,0.69,11,-7.956,1,0.164,0.247,0,0.101,0.497,89.977
Say So - Doja Cat,0.787,0.673,11,-4.577,0,0.158,0.256,3.57e-06,0.0904,0.786,110.962
Everything I Wanted - Billie Eilish,0.704,0.225,6,-14.454,0,0.0994,0.902,0.657,0.106,0.243,120.006
Don’t Start Now - Dua Lipa,0.793,0.793,11,-4.521,0,0.083,0.0123,0,0.0951,0.679,123.95
Circles - Post Malone,0.695,0.762,0,-3.497,1,0.0395,0.192,0.00244,0.0863,0.553,120.042
Savage - Megan Thee Stallion Featuring Beyoncé,0.822,0.743,10,-4.772,0,0.28,0.016,0,0.127,0.636,84.459


In [ ]:
df2

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
Bad Guy – Billie Eilish,0.701,0.425,7,-10.965,1,0.375,0.328,0.13,0.1,0.562,135.128
"Hey, Ma - Bon Iver",0.411,0.285,0,-10.97,1,0.0306,0.872,0.000354,0.0993,0.213,72.139
7 Rings – Ariana Grande,0.778,0.317,1,-10.732,0,0.334,0.592,0,0.0881,0.327,140.048
Hard Place – H.E.R.,0.614,0.719,9,-4.694,1,0.0955,0.179,1.72e-05,0.163,0.34,160.075
Talk – Khalid ft Disclosure,0.901,0.4,0,-8.575,1,0.126,0.051,0,0.0599,0.346,135.984
Old Town Road – Lil Nas X Featuring Billy Ray Cyrus,0.907,0.53,1,-6.112,1,0.127,0.0578,2.23e-06,0.101,0.507,135.998
Truth Hurts – Lizzo,0.713,0.626,4,-2.891,0,0.124,0.111,0,0.126,0.39,158.08
Sunflower – Post Malone & Swae Lee,0.76,0.479,2,-5.574,1,0.0466,0.556,0,0.0703,0.913,89.911


In [ ]:
df3

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
This Is America – Childish Gambino,0.854,0.463,5,-6.159,1,0.137,0.117,0,0.354,0.549,120.024
"I Like It – Cardi B, Bad Bunny & J Balvin",0.816,0.726,5,-3.998,0,0.129,0.099,0,0.372,0.65,136.048
The Joke – Brandi Carlile,0.236,0.578,7,-5.477,1,0.0404,0.327,0.00015,0.206,0.227,75.367
God's Plan – Drake,0.754,0.449,7,-9.211,1,0.109,0.0332,8.29e-05,0.552,0.357,77.169
Shallow – Lady Gaga & Bradley Cooper,0.572,0.385,7,-6.362,1,0.0308,0.371,0,0.231,0.323,95.799
All the Stars – Kendrick Lamar & SZA,0.698,0.633,8,-4.946,1,0.0597,0.0605,0.000194,0.0926,0.552,96.924
Rockstar – Post Malone Featuring 21 Savage,0.585,0.52,5,-6.136,0,0.0712,0.124,7.01e-05,0.131,0.129,159.801
"The Middle – Zedd, Maren Morris and Grey",0.753,0.657,7,-3.061,1,0.0449,0.171,0,0.112,0.437,107.01


In [ ]:
df4

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
That's What I Likea – Bruno Mars,0.853,0.56,1,-4.961,1,0.0406,0.013,0,0.0944,0.86,134.066
Redbone – Childish Gambino,0.743,0.347,1,-11.174,1,0.121,0.167,0.00951,0.103,0.572,160.143
Despacito – Luis Fonsi and Daddy Yankee Featuring Justin Bieber,0.653,0.816,2,-4.353,1,0.167,0.228,0,0.0967,0.816,178.085
The Story of O.J. – Jay-Z,0.741,0.718,7,-5.823,1,0.415,0.283,2.21e-06,0.23,0.576,165.848
Humble – Kendrick Lamar,0.908,0.621,1,-6.638,0,0.102,0.000282,5.39e-05,0.0958,0.421,150.011


In [ ]:
df5

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
Hello – Adele,0.481,0.451,5,-6.095,0,0.0347,0.336,0,0.0872,0.289,157.966
Formation – Beyoncé,0.896,0.621,5,-6.857,0,0.237,0.00532,0,0.196,0.818,121.966
7 Years – Lukas Graham,0.765,0.473,10,-5.829,1,0.0514,0.287,0,0.391,0.34,119.992
Work – Rihanna featuring Drake,0.725,0.534,11,-6.238,1,0.0946,0.0752,0,0.0919,0.558,91.974
Stressed Out – Twenty One Pilots,0.734,0.637,4,-5.677,0,0.141,0.0462,2.29e-05,0.0602,0.648,169.977


In [ ]:
df6

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
Blinding Light - The Weekend,0.514,0.73,1,-5.934,1,0.0598,0.00146,9.54e-05,0.0897,0.334,171.005
Dance Monkey - Tones And I,0.824,0.588,6,-6.4,0,0.0924,0.692,0.000104,0.149,0.513,98.027
The Box - Roddy Ricch,0.896,0.586,10,-6.687,0,0.0559,0.104,0,0.79,0.642,116.971
Roses (Imanbek Remix) - Imanbek and SAINt JHN,0.785,0.721,8,-5.457,1,0.0506,0.0149,0.00432,0.285,0.894,121.962
Don’t Start Now - Dua Lipa,0.793,0.793,11,-4.521,0,0.083,0.0123,0,0.0951,0.679,123.95


In [ ]:
df7

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
"Señorita – Camila Cabello, Shawn Mendes",0.759,0.54,9,-6.039,0,0.0287,0.037,0,0.0945,0.75,116.947
Bad Guy – Billie Eilish,0.701,0.425,7,-10.965,1,0.375,0.328,0.13,0.1,0.562,135.128
"Sunflower – Post Malone, Swae Lee",0.76,0.479,2,-5.574,1,0.0466,0.556,0,0.0703,0.913,89.911
7 Rings – Ariana Grande,0.778,0.317,1,-10.732,0,0.334,0.592,0,0.0881,0.327,140.048
"Old Town Road – Remix – Lil Nas X, Billy Ray Cyrus",0.907,0.53,1,-6.112,1,0.127,0.0578,2.23e-06,0.101,0.507,135.998


In [ ]:
df8

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
God’s Plan – Drake,0.754,0.449,7,-9.211,1,0.109,0.0332,8.29e-05,0.552,0.357,77.169
SAD! – XXXTENTACION,0.74,0.613,8,-4.88,1,0.145,0.258,0.00372,0.123,0.473,75.023
Rockstar – Post Malone Featuring 21 Savage,0.585,0.52,5,-6.136,0,0.0712,0.124,7.01e-05,0.131,0.129,159.801
Psycho – Post Malone Featuring Ty Dolla $ign,0.75,0.56,8,-8.094,1,0.105,0.546,0,0.111,0.459,140.06
In My Feelings – Drake,0.835,0.626,1,-5.833,1,0.125,0.0589,6e-05,0.396,0.35,91.03


In [ ]:
df9

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
Shape of You - Ed Sheeran,0.825,0.652,1,-3.183,0,0.0802,0.581,0,0.0931,0.931,95.977
"Despacito (Remix) - Luis Fonsi, Daddy Yankee and Justin Bieber",0.653,0.816,2,-4.353,1,0.167,0.228,0,0.0967,0.816,178.085
Despacito - Luis Fonsi Featuring Daddy Yankee,0.655,0.797,2,-4.787,1,0.153,0.198,0,0.067,0.839,177.928
Something Just Like This - The Chainsmokers Featuring Coldplay,0.617,0.635,11,-6.769,0,0.0317,0.0498,1.44e-05,0.164,0.446,103.019
"I’m the One - DJ Khaled, Justin Bieber, Chance the Rapper, Quavo, Lil Wayne",0.413,0.807,11,-3.499,0,0.318,0.0536,0,0.631,0.438,149.33


In [ ]:
df10

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
One Dance – Drake Featuring WizKid and Kyla,0.792,0.625,1,-5.609,1,0.0536,0.00776,0.0018,0.329,0.37,103.967
I Took A Pill in Ibiza – Seeb Remix – Mike Posner,0.664,0.714,7,-6.645,0,0.111,0.0353,8.42e-06,0.0843,0.71,101.969
Don’t Let Me Down – The Chainsmokers Featuring Daya,0.532,0.869,11,-5.094,1,0.172,0.157,0.00508,0.136,0.422,159.803
Work – Rihanna Featuring Drake,0.725,0.534,11,-6.238,1,0.0946,0.0752,0,0.0919,0.558,91.974
Cheap Thrills – Sia,0.628,0.698,6,-5.608,0,0.105,0.0472,0.00143,0.0907,0.732,89.976


In [ ]:
df1.to_csv(r'C://Users/samue/Desktop/Grammy2020.csv',index=True)
df2.to_csv(r'C://Users/samue/Desktop/Grammy2019.csv',index=True)
df3.to_csv(r'C://Users/samue/Desktop/Grammy2018.csv',index=True)
df4.to_csv(r'C://Users/samue/Desktop/Grammy2017.csv',index=True)
df5.to_csv(r'C://Users/samue/Desktop/Grammy2016.csv',index=True)
df6.to_csv(r'C://Users/samue/Desktop/Spotify2020.csv',index=True)
df7.to_csv(r'C://Users/samue/Desktop/Spotify2019.csv',index=True)
df8.to_csv(r'C://Users/samue/Desktop/Spotify2018.csv',index=True)
df9.to_csv(r'C://Users/samue/Desktop/Spotify2017.csv',index=True)
df10.to_csv(r'C://Users/samue/Desktop/Spotify2016.csv',index=True)

In [ ]:
import plotly.express as px 
import pandas as pd

df = pd.read_excel('/content/AllCompiled.xlsx') 
fig = px.parallel_coordinates(df,color="Platform",color_continuous_scale=px.colors.diverging.RdYlBu)
  
fig.show()

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

iplot(fig, image='svg', filename='scatterplot',image_width=1600, image_height=525)

In [ ]:
import plotly.express as px 
import pandas as pd

df = pd.read_csv('/content/Artist.csv') 
fig = px.parallel_categories(df,color='Platform',color_continuous_scale=px.colors.diverging.RdYlBu)
  
fig.show()

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

iplot(fig, image='svg', filename='scatterplot',image_width=1600, image_height=525)